<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/Whatsapp_Society_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**⏯ Authorization & Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
import dateutil.parser
import re
from datetime import date

# from dateutil.relativedelta import relativedelta
from functools import reduce
from datetime import datetime
from datetime import date, timedelta

from google.colab import drive
drive.mount('/content/drive', force_remount = True)


import gspread
from google.colab import auth
# auth.authenticate_user()

# from oauth2client.client import GoogleCredentials
# gc = gspread.authorize(GoogleCredentials.get_application_default())
from gspread_dataframe import set_with_dataframe

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

Mounted at /content/drive


In [ ]:
def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def parseDateTimeFormatTZ(row):
  return dateutil.parser.parse(row)

def cleanText(row):
  return row.str.upper().str.strip()

# def getDump(mastermerge, dump_name,start_date, end_date):
#   start_date = pd.to_datetime(start_date).date()
#   end_date = pd.to_datetime(end_date).date()
#   file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
#   reg = re.compile(".+" + dump_name+".csv")

#   for (root, dirs, files) in os.walk(file_path):
#       for file in files:
#         path = os.path.join(root, file)
#         date = str(path).split('/')[7:][0]
#         date = pd.to_datetime(date).date()

#         if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
#           temp = pd.read_csv(path,parse_dates=True)
#           temp['DumpDate'] = date
#           read_dump_name = str(path).split('/')[9:][0]
#           print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
#           mastermerge = mastermerge.append(temp)

#   mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
#   return mastermerge

Importing Dumps

In [ ]:
start = "2023-01-02"
end = "2023-01-08"
file_path = '/content/drive/MyDrive/Whatsapp Pulse Report Dumps/'

In [ ]:
visit_report = pd.read_csv(file_path+"Visit Report.csv")
visit_report["society_id"] = visit_report['society_id'].apply(lambda x: x.strip())
bill_publish = pd.read_csv(file_path+"Bill Publish Date - Society wise.csv")
bill_publish = bill_publish[(bill_publish['Approved Date']>=start) & (bill_publish['Approved Date']<=end)]
bill_publish = bill_publish[['Society Name','Society ID','Approved Date','Bill Types','Total Amount','Valid Bill Count']]
transaction = pd.read_csv(file_path+"Transactions (last 30 days).csv")
transaction["Society ID"] = transaction['Society ID'].apply(lambda x: x.strip())
complaint = pd.read_csv(file_path+"Complaint.csv")
complaint['Resolved'] = complaint['Count'] - complaint['Not Resolved']
notice_board = pd.read_csv(file_path+"Notice Board.csv")
notice_board = notice_board[['Society','Society ID','Last 7 days','Last 30 days']]
notice_board['Society ID'] = notice_board['Society ID'].apply(lambda x: x.strip())
forum = pd.read_csv(file_path+"Forum Usage.csv")
forum['Society ID'] = forum['Society ID'].apply(lambda x: x.strip())
unchurned = pd.read_csv(file_path+"Auto Bill Generation.csv")
unchurned["society_id"] = unchurned['society_id'].apply(lambda x: x.strip())
unchurned['Date'] = unchurned['Auto Bill Generation Day'].apply(lambda x: str(x).split("T")[0])
unchurned

,society_name,society_id,bill_type,Auto Bill Generation Day,Date
0,Harmony Apartment Owners Welfare Society,8a96a4a07a4e5086017a53abfa8f5b9e,MAINTENANCE,2021-10-01T00:00:00+05:30,2021-10-01
1,Harmony Apartment Owners Welfare Society,8a96a4a07a4e5086017a53abfa8f5b9e,MAINTENANCE,2021-11-01T00:00:00+05:30,2021-11-01
2,Harmony Apartment Owners Welfare Society,8a96a4a07a4e5086017a53abfa8f5b9e,MAINTENANCE,2021-12-01T00:00:00+05:30,2021-12-01
3,Harmony Apartment Owners Welfare Society,8a96a4a07a4e5086017a53abfa8f5b9e,MAINTENANCE,2022-01-01T00:00:00+05:30,2022-01-01
4,Harmony Apartment Owners Welfare Society,8a96a4a07a4e5086017a53abfa8f5b9e,MAINTENANCE,2022-02-01T00:00:00+05:30,2022-02-01
...,...,...,...,...,...
2477,v2 poorvi enclave,8a969fd17d6c8b86017d6f24b8fc195e,MAINTENANCE_CHARGES,2022-08-01T00:00:00+05:30,2022-08-01
2478,v2 poorvi enclave,8a969fd17d6c8b86017d6f24b8fc195e,MAINTENANCE_CHARGES,2022-09-01T00:00:00+05:30,2022-09-01
2479,v2 poorvi enclave,8a969fd17d6c8b86017d6f24b8fc195e,MAINTENANCE_CHARGES,2022-10-01T00:00:00+05:30,2022-10-01
2480,v2 poorvi enclave,8a969fd17d6c8b86017d6f24b8fc195e,MAINTENANCE_CHARGES,2022-11-01T00:00:00+05:30,2022-11-01


In [ ]:
# from gspread_dataframe import set_with_dataframe
# spreadSheetId = "1Zt3sekf7sHkqRQz-A4vTSBdKYm0MyUGuYLT4Vlz5h-A"
# rangeName = "data_dumps"
# wb = gc.open_by_key(spreadSheetId)
# ws = wb.worksheet(rangeName)
# ws.clear()
# ws.update('A:F',visit_report)
# # df = pd.DataFrame()
# # set_with_dataframe(ws, df)

In [ ]:
#Master data

wb = gc.open_by_key("1iKgiRR-lOQ-zM6v6bh_1P3vkIujVCeBzVVr_nHuoDMU")
ws = wb.worksheet("Society Master")
master_data = ws.get_all_values()
master_data = pd.DataFrame(master_data)
master_data.columns = master_data.iloc[0]
master_data = master_data.drop(0)
master_data = master_data[(master_data['societyStatus']=='ACTIVE') | ((master_data['societyStatus']=='YTB') & (master_data['Original Closure Number'].isin([1024849,909])))]
master_data = master_data[['Society Name','Kibana ID','Original Closure Number','City','KAM Onboarding','Sold Flats','Downloads','ERP Onb Status','VMS Onb Status']]
master_data[['Downloads','Sold Flats']] = master_data[['Downloads','Sold Flats']].apply(pd.to_numeric)
master_data['Download%'] = master_data.apply(lambda x: 1 if (x['Downloads']/x['Sold Flats'])>1 else (x['Downloads']/x['Sold Flats']),axis='columns').fillna(0) *100
master_data['Final App Download %'] = master_data.apply(lambda x: x['Download%'] if len(x['Society Name'])>0 else "",axis='columns')
total_entry = visit_report.groupby('society_id').agg({'Count':'sum'}).reset_index().fillna(0)
preapproved_entry = visit_report[visit_report['approval_type']=='PRE_APPROVED'].groupby('society_id').agg({'Count':'sum'}).reset_index().fillna(0)

master_data = pd.merge(master_data,total_entry,how='left',left_on='Kibana ID', right_on = 'society_id')
master_data = pd.merge(master_data,preapproved_entry,how='left',left_on='Kibana ID', right_on = 'society_id')
# pre_approved = visit_report[visit_report['approval_type']=='PRE_APPROVED']
master_data = master_data.rename(columns = {'Count_x':'Total Entry','Count_y':'Pre-approved Entries'})
master_data = master_data[['Society Name','Kibana ID','Original Closure Number','City','KAM Onboarding','Sold Flats','Downloads','ERP Onb Status','VMS Onb Status','Download%','Final App Download %','Total Entry','Pre-approved Entries']]
master_data['Entries / Flat'] = master_data['Total Entry']/master_data['Downloads']
master_data['Pre-Approved %'] = (master_data['Pre-approved Entries']/master_data['Total Entry'])*100
master_data = pd.merge(master_data,forum[['Society ID','Topics Created Last 7 days','Unique Users Read in Last 7 days']],how='left',left_on='Kibana ID', right_on = 'Society ID').fillna(0)
master_data = pd.merge(master_data,complaint[['party → id','Count','Resolved']],how='left',left_on='Kibana ID', right_on = 'party → id')
master_data['Complaints Resolution'] = (master_data['Resolved']/master_data['Count'])*100
master_data = pd.merge(master_data,notice_board[['Society ID','Last 7 days']],how='left',left_on='Kibana ID', right_on = 'Society ID')

master_data
# preapproved_entry

,Society Name,Kibana ID,Original Closure Number,City,KAM Onboarding,Sold Flats,Downloads,ERP Onb Status,VMS Onb Status,Download%,...,Pre-Approved %,Society ID_x,Topics Created Last 7 days,Unique Users Read in Last 7 days,party → id,Count,Resolved,Complaints Resolution,Society ID_y,Last 7 days
0,ORCHID LAKEVIEW OWNERS ASSOCIATION,8a96a79d85787a7f01857b29883458c5,1029266,BANGALORE,mallikarjuna.s1@nobroker.in,336.0,302.0,ON-BOARDED,YTB,89.880952,...,3.099174,8a96a79d85787a7f01857b29883458c5,1.0,20.0,8a96a79d85787a7f01857b29883458c5,9.0,5.0,55.555556,8a96a79d85787a7f01857b29883458c5,6.0
1,LAXMI EMERALD,8a96a79b856e2d440185726b47ff5078,1029174,PUNE,pranesh.gaikwad@nobroker.in,78.0,67.0,NOT SOLD,ON-BOARDED,85.897436,...,0.000000,8a96a79b856e2d440185726b47ff5078,1.0,3.0,NaN,NaN,NaN,NaN,8a96a79b856e2d440185726b47ff5078,1.0
2,SHREE BALAJI RESIDENCY,8a96a7978559942f01855d8161304585,1029025,AHMEDABAD,shaikh.salman@nobroker.in,90.0,56.0,NOT SOLD,ON-BOARDED,62.222222,...,0.272480,0,0.0,0.0,8a96a7978559942f01855d8161304585,1.0,0.0,0.000000,8a96a7978559942f01855d8161304585,1.0
3,GOTHIC PANGEA,8a96a798855ebaa001856260ede52ae9,1029157,HYDERABAD,theneti.ravicharan1@nobroker.in,59.0,43.0,NOT SOLD,ON-BOARDED,72.881356,...,5.235602,8a96a798855ebaa001856260ede52ae9,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,PARK ROYAL,8a96a7978559942f01855e55d71d03e0,7513,HYDERABAD,bethi.kiran@nobroker.in,186.0,134.0,ON-BOARDED,ON-BOARDED,72.043011,...,1.229508,8a96a7978559942f01855e55d71d03e0,0.0,11.0,8a96a7978559942f01855e55d71d03e0,6.0,4.0,66.666667,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,RWA 21A EAST,8a2080ac744d409d01744dc46a96730e,4063,Gurgaon,,286.0,8.0,PHASE 1 CHURN,ON HOLD,2.797203,...,0.000000,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
9354,SUNSHINE APARTMENT,8a968fc974c5c3ed0174d05a003460f8,4506,Gurgaon,,69.0,6.0,PHASE 1 CHURN,ON HOLD,8.695652,...,0.000000,8a968fc974c5c3ed0174d05a003460f8,0.0,0.0,NaN,NaN,NaN,NaN,8a968fc974c5c3ed0174d05a003460f8,1.0
9355,AJAY ENCLAVE,8a968eea75033e0c017507948088500a,4857,Gurgaon,,95.0,8.0,PHASE 1 CHURN,ON HOLD,8.421053,...,0.000000,0,0.0,0.0,NaN,NaN,NaN,NaN,8a968eea75033e0c017507948088500a,1.0
9356,EDEN TOWER (ATLANTA 6),8a968ff3750bcc5601750c01bdde3267,4795.1,Navi Mumbai - Thane,kunal.gidh@nobroker.in,48.0,0.0,PHASE 1 CHURN,ON HOLD,0.000000,...,0.000000,8a968ff3750bcc5601750c01bdde3267,0.0,0.0,NaN,NaN,NaN,NaN,8a968ff3750bcc5601750c01bdde3267,1.0


In [ ]:
transaction = transaction[transaction['Payment Mode']=='Online'].groupby('Society ID').agg({'Unique Flats':'sum','Transactions':'sum','Amount':'sum'}).reset_index()
master_data = pd.merge(master_data,transaction[['Society ID','Unique Flats','Transactions','Amount']],how='left',left_on='Kibana ID', right_on = 'Society ID')


In [ ]:
unchurned = unchurned[['society_id','Date']].groupby('society_id').apply(lambda x: "YES" if x['Date'].count()>0 else "NO")
unchurned = pd.DataFrame({'Society ID':unchurned.index,'Auto-bill':unchurned.values})
master_data = pd.merge(master_data,unchurned,how='left',left_on='Kibana ID', right_on = 'Society ID')
master_data['Auto-bill']=master_data['Auto-bill'].fillna("NO")
master_data.columns

<ipython-input-28-8b729ca23cad>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Society ID_x'} in the result is deprecated and will raise a MergeError in a future version.
  master_data = pd.merge(master_data,unchurned,how='left',left_on='Kibana ID', right_on = 'Society ID')


Index(['Society Name', 'Kibana ID', 'Original Closure Number', 'City',
       'KAM Onboarding', 'Sold Flats', 'Downloads', 'ERP Onb Status',
       'VMS Onb Status', 'Download%', 'Final App Download %', 'Total Entry',
       'Pre-approved Entries', 'Entries / Flat', 'Pre-Approved %',
       'Society ID_x', 'Topics Created Last 7 days',
       'Unique Users Read in Last 7 days', 'party → id', 'Count', 'Resolved',
       'Complaints Resolution', 'Society ID_y', 'Last 7 days', 'Society ID_x',
       'Unique Flats', 'Transactions', 'Amount', 'Society ID_y', 'Auto-bill'],
      dtype='object')

In [ ]:
master_data['Online Transaction %'] = (master_data['Transactions']/master_data['Downloads'])*100
master_data

,Society Name,Kibana ID,Original Closure Number,City,KAM Onboarding,Sold Flats,Downloads,ERP Onb Status,VMS Onb Status,Download%,...,Complaints Resolution,Society ID_y,Last 7 days,Society ID_x,Unique Flats,Transactions,Amount,Society ID_y,Auto-bill,Online Transaction %
0,ORCHID LAKEVIEW OWNERS ASSOCIATION,8a96a79d85787a7f01857b29883458c5,1029266,BANGALORE,mallikarjuna.s1@nobroker.in,336.0,302.0,ON-BOARDED,YTB,89.880952,...,55.555556,8a96a79d85787a7f01857b29883458c5,6.0,8a96a79d85787a7f01857b29883458c5,26.0,26.0,430155.0,NaN,NO,8.609272
1,LAXMI EMERALD,8a96a79b856e2d440185726b47ff5078,1029174,PUNE,pranesh.gaikwad@nobroker.in,78.0,67.0,NOT SOLD,ON-BOARDED,85.897436,...,NaN,8a96a79b856e2d440185726b47ff5078,1.0,NaN,NaN,NaN,NaN,NaN,NO,NaN
2,SHREE BALAJI RESIDENCY,8a96a7978559942f01855d8161304585,1029025,AHMEDABAD,shaikh.salman@nobroker.in,90.0,56.0,NOT SOLD,ON-BOARDED,62.222222,...,0.000000,8a96a7978559942f01855d8161304585,1.0,NaN,NaN,NaN,NaN,NaN,NO,NaN
3,GOTHIC PANGEA,8a96a798855ebaa001856260ede52ae9,1029157,HYDERABAD,theneti.ravicharan1@nobroker.in,59.0,43.0,NOT SOLD,ON-BOARDED,72.881356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN
4,PARK ROYAL,8a96a7978559942f01855e55d71d03e0,7513,HYDERABAD,bethi.kiran@nobroker.in,186.0,134.0,ON-BOARDED,ON-BOARDED,72.043011,...,66.666667,NaN,NaN,8a96a7978559942f01855e55d71d03e0,27.0,35.0,537375.0,NaN,NO,26.119403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,RWA 21A EAST,8a2080ac744d409d01744dc46a96730e,4063,Gurgaon,,286.0,8.0,PHASE 1 CHURN,ON HOLD,2.797203,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN
9354,SUNSHINE APARTMENT,8a968fc974c5c3ed0174d05a003460f8,4506,Gurgaon,,69.0,6.0,PHASE 1 CHURN,ON HOLD,8.695652,...,NaN,8a968fc974c5c3ed0174d05a003460f8,1.0,NaN,NaN,NaN,NaN,NaN,NO,NaN
9355,AJAY ENCLAVE,8a968eea75033e0c017507948088500a,4857,Gurgaon,,95.0,8.0,PHASE 1 CHURN,ON HOLD,8.421053,...,NaN,8a968eea75033e0c017507948088500a,1.0,NaN,NaN,NaN,NaN,NaN,NO,NaN
9356,EDEN TOWER (ATLANTA 6),8a968ff3750bcc5601750c01bdde3267,4795.1,Navi Mumbai - Thane,kunal.gidh@nobroker.in,48.0,0.0,PHASE 1 CHURN,ON HOLD,0.000000,...,NaN,8a968ff3750bcc5601750c01bdde3267,1.0,NaN,NaN,NaN,NaN,NaN,NO,NaN


In [ ]:
bill_publish = bill_publish.groupby('Society ID').sum().reset_index()
master_data = pd.merge(master_data,bill_publish[['Society ID','Total Amount','Valid Bill Count']],how='left',left_on='Kibana ID', right_on = 'Society ID')
master_data['Manual Transaction %'] = master_data['Unique Flats']/master_data['Downloads']*100



In [ ]:
visitor_entry = visit_report[~visit_report['visitor_type'].isin(['SERVICES','STAFF'])].groupby('society_id').agg({'Count':'sum'}).reset_index().fillna(0)
approved_visitor = visit_report[(~visit_report['visitor_type'].isin(['SERVICES','STAFF'])) & (visit_report['approval_type'].isin(['RESIDENT','PRE_APPROVED','INTERCOM','DENIED_FINAL_RESIDENT']))].groupby('society_id').agg({'Count':'sum'}).reset_index().fillna(0)
master_data = pd.merge(master_data,visitor_entry,how='left',left_on='Kibana ID', right_on = 'society_id')
master_data = pd.merge(master_data,approved_visitor,how='left',left_on='Kibana ID', right_on = 'society_id')
master_data.columns

Index(['Society Name', 'Kibana ID', 'Original Closure Number', 'City',
       'KAM Onboarding', 'Sold Flats', 'Downloads', 'ERP Onb Status',
       'VMS Onb Status', 'Download%', 'Final App Download %', 'Total Entry',
       'Pre-approved Entries', 'Entries / Flat', 'Pre-Approved %',
       'Society ID_x', 'Topics Created Last 7 days',
       'Unique Users Read in Last 7 days', 'party → id', 'Count_x', 'Resolved',
       'Complaints Resolution', 'Society ID_y', 'Last 7 days', 'Society ID_x',
       'Unique Flats', 'Transactions', 'Amount', 'Society ID_y', 'Auto-bill',
       'Online Transaction %', 'Society ID', 'Total Amount',
       'Valid Bill Count', 'Manual Transaction %', 'society_id_x', 'Count_y',
       'society_id_y', 'Count'],
      dtype='object')

In [ ]:
master_data.drop(columns = ['Society ID_x','Society ID_x','society_id_y','Society ID_y','Society ID_y','society_id_x','party → id'], inplace=True)
master_data.columns

Index(['Society Name', 'Kibana ID', 'Original Closure Number', 'City',
       'KAM Onboarding', 'Sold Flats', 'Downloads', 'ERP Onb Status',
       'VMS Onb Status', 'Download%', 'Final App Download %', 'Total Entry',
       'Pre-approved Entries', 'Entries / Flat', 'Pre-Approved %',
       'Topics Created Last 7 days', 'Unique Users Read in Last 7 days',
       'Count_x', 'Resolved', 'Complaints Resolution', 'Last 7 days',
       'Unique Flats', 'Transactions', 'Amount', 'Auto-bill',
       'Online Transaction %', 'Society ID', 'Total Amount',
       'Valid Bill Count', 'Manual Transaction %', 'Count_y', 'Count'],
      dtype='object')

In [ ]:
master_data = master_data.rename(columns = {'Topics Created Last 7 days':'Forums Posted','Unique Users Read in Last 7 days':'Forums Read','Count_x':'Complaints Raised','Resolved':'Complaints Resolved','Last 7 days':'Notices','Unique Flats':'Residents Using Transaction (online)',\
                              'Transactions':'MTD Transaction (online)','Total Amount':'Bill Posted amount','Valid Bill Count':'Bill posted - valid bill count','Amount':'Transaction - online Amount','Count_y':'Total Visitor entries (excluding STAFF & SERVICES)',\
                              'Count':'Total Visitor entries (excluding Staff and Services) - resident approved'})

In [ ]:
master_data['Total Visitor entries (excluding Staff) - resident approved %'] = (master_data['Total Visitor entries (excluding Staff and Services) - resident approved']/master_data['Total Visitor entries (excluding STAFF & SERVICES)'])*100

In [ ]:
master_data

,Society Name,Kibana ID,Original Closure Number,City,KAM Onboarding,Sold Flats,Downloads,ERP Onb Status,VMS Onb Status,Download%,...,Transaction - online Amount,Auto-bill,Online Transaction %,Society ID,Bill Posted amount,Bill posted - valid bill count,Manual Transaction %,Total Visitor entries (excluding STAFF & SERVICES),Total Visitor entries (excluding Staff and Services) - resident approved,Total Visitor entries (excluding Staff) - resident approved %
0,ORCHID LAKEVIEW OWNERS ASSOCIATION,8a96a79d85787a7f01857b29883458c5,1029266,BANGALORE,mallikarjuna.s1@nobroker.in,336.0,302.0,ON-BOARDED,YTB,89.880952,...,430155.0,NO,8.609272,8a96a79d85787a7f01857b29883458c5,5632542.0,337.0,8.609272,436.0,289.0,66.284404
1,LAXMI EMERALD,8a96a79b856e2d440185726b47ff5078,1029174,PUNE,pranesh.gaikwad@nobroker.in,78.0,67.0,NOT SOLD,ON-BOARDED,85.897436,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SHREE BALAJI RESIDENCY,8a96a7978559942f01855d8161304585,1029025,AHMEDABAD,shaikh.salman@nobroker.in,90.0,56.0,NOT SOLD,ON-BOARDED,62.222222,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,292.0,178.0,60.958904
3,GOTHIC PANGEA,8a96a798855ebaa001856260ede52ae9,1029157,HYDERABAD,theneti.ravicharan1@nobroker.in,59.0,43.0,NOT SOLD,ON-BOARDED,72.881356,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,169.0,131.0,77.514793
4,PARK ROYAL,8a96a7978559942f01855e55d71d03e0,7513,HYDERABAD,bethi.kiran@nobroker.in,186.0,134.0,ON-BOARDED,ON-BOARDED,72.043011,...,537375.0,NO,26.119403,8a96a7978559942f01855e55d71d03e0,4500.0,3.0,20.149254,167.0,99.0,59.281437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9353,RWA 21A EAST,8a2080ac744d409d01744dc46a96730e,4063,Gurgaon,,286.0,8.0,PHASE 1 CHURN,ON HOLD,2.797203,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9354,SUNSHINE APARTMENT,8a968fc974c5c3ed0174d05a003460f8,4506,Gurgaon,,69.0,6.0,PHASE 1 CHURN,ON HOLD,8.695652,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9355,AJAY ENCLAVE,8a968eea75033e0c017507948088500a,4857,Gurgaon,,95.0,8.0,PHASE 1 CHURN,ON HOLD,8.421053,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9356,EDEN TOWER (ATLANTA 6),8a968ff3750bcc5601750c01bdde3267,4795.1,Navi Mumbai - Thane,kunal.gidh@nobroker.in,48.0,0.0,PHASE 1 CHURN,ON HOLD,0.000000,...,NaN,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
spreadSheetId = "14yUHikTUcTKQ5NeMLM5UD11tzHGg3Lz4_voa0T8Y1ek"
rangeName = "Sheet1"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,master_data)